## **Wavelet steganography encode (only on binary images)**



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import pywt.data
from PIL import Image
import math

Breaking the image into four parts using integer wavelet transformation ::: 

In [3]:
def for_int_to_int(img):
    l=list(img.getdata())
    w=img.size[0]
    h=img.size[1]
    new_data=[]

    for i in range(h):
        row=[]
        for j in range(w):
            row.append(int(l[(w*i)+j]))
        
        length=math.floor(int(len(row))/2)
        a=[0]*length
        d=[0]*length
    
        for k in range(length):
            a[k]=math.floor((row[2*k]+row[(2*k)+1])/2)
            d[k]=row[2*k]-row[(2*k)+1]
        for y in range(length):
            new_data.append(int(a[y]))
        for y in range(length):
            new_data.append(int(d[y]))


    for i in range(h):
        for j in range(w):
            img.putpixel((j,i),int(new_data[(i*w)+j]))
                      
    img.save('row_frwrd.png')# result of low and high pass filter row wise
    img=Image.open('row_frwrd.png')       
    l=list(img.getdata())

    for i in range(w):
        col=[]
        for j in range(h):
            col.append(int(l[(w*j)+i]))
        
        length=math.floor(int(len(col))/2)
        a=[0]*length
        b=[0]*length
    
        for k in range(length):
            a[k]=math.floor((col[2*k]+col[(2*k)+1])/2)
            d[k]=col[2*k]-col[(2*k)+1]
        for u in range(length):
            img.putpixel((i,u),int(a[u]))
            img.putpixel((i,u+length),int(d[u]))   

    img.save('w.png') # in 'w.png' we can see the four parts of the macao image after the wavelet transformation(got by applying low and high pas filter on 'row_frwrd')

encoding the data using simple lsb method into the l-l(low-low) component of the image :::

In [4]:
def lsb_steg(datalist,data):
    l=format(len(data),'08b')
    for i in range(-8,0):
        if int(l[i])==1 and datalist[i]%2==0:
            datalist[i]=datalist[i]+1
        elif int(l[i])==0 and datalist[i]%2==1:
            datalist[i]=datalist[i]+1
            
    for j in range(len(data)):
        l=format(ord(data[j]),'08b')       
        for i in range(8):
            if int(l[i])==1 and datalist[j*8+i]%2==0:
                datalist[j*8+i]=datalist[j*8+i]+1
            elif int(l[i])==0 and datalist[j*8+i]%2==1:
                datalist[j*8+i]=datalist[j*8+i]+1

reconstruction of the image after encoding the data :::

In [5]:
def inv_int_to_int(img):
    w=img.size[0]
    h=img.size[1]
    l=list(img.getdata())

    for i in range(w):
        col=[]
        for j in range(h):
            col.append(int(l[(w*j)+i]))
            
        length=math.floor(int(len(col))/2)
        a=[0]*length
        d=[0]*length
        for u in range(length):
            a[u]=col[u]
            d[u]=col[u+length]
        x=[0]*len(col)
        for k in range(length):
            x[2*k]=int(a[k])+math.floor((int(d[k])+1)/2)
            x[(2*k)+1]=int(x[2*k])-int(d[k])
            
        for v in range(h):
            img.putpixel((i,v),int(x[v]))
    img.save('col_bkwrd.png')        
    
    img=Image.open('col_bkwrd.png')
    l=list(img.getdata())
    for i in range(h):
        row=[]
        for j in range(w):
            row.append(int(l[(i*w)+j]))
        
        length=math.floor(int(len(row))/2)
        a=[0]*length
        d=[0]*length
        for u in range(length):
            a[u]=row[u]
            d[u]=row[length+u]
        x=[0]*len(row)
        for k in range(length):
            x[2*k]=int(a[k])+math.floor((int(d[k])+1)/2)
            x[(2*k)+1]=int(x[2*k])-int(d[k])
        
        for v in range(w):
            img.putpixel((v,i),int(x[v]))
    
    
    img.save('winv.png')

main function :::

In [11]:
def main(image):
    img=Image.open(image)
    data='nilabjanayan bera'
    
    for_int_to_int(img)
    nimg=Image.open('w.png')
    datalist=[]
    l=list(nimg.getdata())
    w=nimg.size[0]
    h=nimg.size[1]
    for i in range(math.floor(h/2)):
        for j in range(math.floor(w/2),w):
            datalist.append(l[(i*w)+j])       
    
    lsb_steg(datalist,data)
    
    key=0
    for i in range(math.floor(h/2)):
        for j in range(math.floor(w/2),w):
            nimg.putpixel((j,i),datalist[key])
            key=key+1
    nimg.save('w1.png')               
    
    img=Image.open('w1.png')
    inv_int_to_int(img)
    

    

In [13]:
main('macao.png')